<hr/>
<center>探究不同性格间文本内容的相似度</center>
<hr/>

In [1]:
import datasets

DataFilePath = "./personality_dataset"

In [2]:
def get_data(path):
    train = datasets.load_from_disk(path+"/train")
    return train

train = get_data(DataFilePath)

In [3]:
labels = ["ENTJ", "ENTP", "ENFJ", "ENFP", "ESFJ", "ESFP", "ESTJ", "ESTP", "INTP", "INTJ", "INFP", "INFJ", "ISFP", "ISFJ", "ISTP", "ISTJ"]

In [4]:
nums = [0 for i in range(16)]

for person in train["personality"]:
    nums[labels.index(person)] += 1

nums

[1934,
 4456,
 2803,
 5186,
 552,
 758,
 494,
 638,
 7213,
 6913,
 10862,
 9281,
 2128,
 1923,
 2164,
 1667]

In [10]:
import tqdm


# 选取两个数量相近的性格
personality = ["ENTJ", "ISFP"]
ENTJ = [train[i]["content"].replace("|||", ".") for i in tqdm.tqdm(range(len(train)), position=0) if train[i]["personality"] == "ENTJ"]
ISFP = [train[i]["content"].replace("|||", ".") for i in tqdm.tqdm(range(len(train)), position=0) if train[i]["personality"] == "ISFP"]

100%|██████████| 58972/58972 [00:02<00:00, 20281.49it/s]


In [19]:
import spacy

# 做词数统计
nlp = spacy.load("en_core_web_sm", exclude=["tok2vec", "tagger", "senter", "attribute_ruler", "lemmatizer"])
words1 = {}
words2 = {}
words = {}
for i in tqdm.tqdm(range(len(ENTJ)), position=0):
    ENTJ[i] = nlp(ENTJ[i])
    for token in ENTJ[i]:
        if token.text in words1.keys():
            words1[token.text] += 1
            words[token.text] += 1
        else:
            words[token.text] = 1
            words1[token.text] = 1
            
for i in tqdm.tqdm(range(len(ISFP)), position=0):
    ISFP[i] = nlp(ISFP[i])
    for token in ISFP[i]:
        if token.text in words2.keys():
            words2[token.text] += 1
        else:
            words2[token.text] = 1
        if token.text in words.keys():
            words[token.text] += 1
        else:
            words[token.text] = 1

100%|██████████| 2128/2128 [00:56<00:00, 37.54it/s]


In [50]:
# 预处理后期统一数据维度
lst1 = [words1[i] if i in words1.keys() else 0 for i in words.keys()]
lst2 = [words2[i] if i in words2.keys() else 0 for i in words.keys()]

In [63]:
import numpy as np


# 得到词数的编码向量
fea1 = np.array(lst1)
fea2 = np.array(lst2)

In [64]:
# 计算每一种词汇所占比例
fea1 = fea1 / np.sum(fea1)
fea2 = fea2 / np.sum(fea2)

In [65]:
# 变为单位向量
fea1 = fea1 / np.linalg.norm(fea1)
fea2 = fea2 / np.linalg.norm(fea2)

In [66]:
cor = fea1.T.dot(fea2)

In [67]:
cor

0.9852139611568004